<a href="https://colab.research.google.com/github/Morningstar1998/GCN_conv_emotion/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clonamos el repositorio

In [ ]:
!git clone https://github.com/Morningstar1998/GCN_conv_emotion

Cloning into 'GCN_conv_emotion'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 70 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (70/70), done.


Instalamos las dependencias necesarias para colab (No es necesario si ya se tiene el  requirements intalado)

In [ ]:
!pip install torch-scatter #-f https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install torch-sparse #-f https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install torch-cluster #-f #https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install torch-spline-conv #-f https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install torch-geometric
!pip install transformers
!pip install sentencepiece

Anexamos los .py donde se encuentra el modelo a la libreria

In [ ]:
import sys
lib='/content/GCN_conv_emotion/GCN'
sys.path.insert(0,lib)

Importamos las librerias 

In [ ]:
import model
import sys
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from dataloader import IEMOCAPDataset
from model import MaskedNLLLoss, LSTMModel, GRUModel, DialogRNNModel, DialogueGCNModel
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report, precision_recall_fscore_support
import pickle
import torchtext
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import torch
import torch.nn as nn
from torchtext.vocab import GloVe
from functools import reduce
import torch as T

Para usar un modelo especifico sobre escribir el path con el modelo deseado que se encuente en el github o de manera local

In [ ]:
path='/content/GCN_conv_emotion/models/GNCGraphDialogRNN_4.pth'
model = torch.load(path)
model.eval()

Esto es para extender lo que se puede ver con colab

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

Visualizamos el pickle con los features guardados 

In [ ]:
x=pickle.load(open('/content/GCN_conv_emotion/GCN/IEMOCAP_features/IEMOCAP_features.pkl', 'rb'), encoding='latin1')
x[3]

Vemos como funciona glove (El cual ya esta tanto importado pero falta instalarlo) Para esta version usaremos el que da 300 como dimension


embedding_glove = GloVe(name='840B', dim=300)


In [ ]:
embedding_glove = GloVe(name='840B', dim=300)

In [ ]:
print(embedding_glove['beatiful'].size())
print(embedding_glove['beatiful'])
c=embedding_glove['beatiful']

Tokenizamos y pasamos a un tensor usando GloVe y la oracion para despues concatenarla en un solo tensor

In [ ]:
text = "Hello my friends! How are you doing today?"
tokenizer = get_tokenizer('basic_english')
text_token=tokenizer(text)
data=[]
for x in text_token:
  data.append(embedding_glove[x])
print(data[1].size())
y=torch.empty 
datafinal=reduce(lambda x,y: T.cat((x,y)), data[:-1])


In [ ]:
print(data[2])

In [ ]:
datafinal

Dado que el modelo de prediccion no funciono puesto que no explican de manera contundente como realizar una inferencia fuera del set de datos. Trataremos de bypasearlo . El modelo pide 3 datos adicionales al input umask , qmask y lenght. Analizado el tipo de datos....

In [ ]:
print(datafinal.size())
umask=torch.tensor(1)
qmask=torch.tensor([1,0])
l=[1]

In [ ]:
try:
  output = model(datafinal,umask,qmask)
  _, predicted = torch.max(output, 1)
except:
  print("Error")

# TRAINING

---
Para entrenar los modelos y ver su desempeño basta con dar correr la celda. Si se desea correr el modelo y guardarlo se necesita ingresar al /content GCN_conv_emotion/GCN/train_IEMOCAP.py y descomentar las lineas 348 si se usa convolucion grafica o la 362 si se usa solo el modelo base en caso de que no se requiera guardar comentar dichas lineas evitara que se guarde.

---

El repositorio cuenta con 4 modelos base: GRU , Linear , LTSM y RNN

Se puede combinar con una convulsion grafica

---

Los mejores resultados se obtienen con epochs entre 50 y 60. No se puede usar GPU por cuestiones de colab

---
De manera adicional, si no se ejecuta en colab sera necesario cambiar  la linea 11 del dataloader.py (GCN_conv_emotion/GCN/dataloader.py) y cambiar el archivo pickle por su nueva direccion (el archivo se encuentra GCN_conv_emotion/GCN/IEMOCAP_features) y la linea 338 y 352 del train_IEMOCAP.py (GCN_conv_emotion/GCN/train_IEMOCAP.py) por un path nuevo donde se se guste guardar los modelos nuevos.


In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --active-listener --epoch 5 --base-model 'GRU' --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --active-listener --epoch 5 --base-model 'DialogRNN' --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --active-listener --epoch 5 --base-model 'LSTM' --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --epoch 5 --base-model 'LSTM' --graph-model --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --epoch 5 --base-model 'DialogRNN' --graph-model --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --epoch 5 --base-model 'GRU' --graph-model --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --epoch 5 --base-model 'None' --graph-model --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

In [ ]:
#Entrenamiento original, con este se obtienen los resultados del paper
!python /content/GCN_conv_emotion/GCN/train_IEMOCAP.py --base-model 'LSTM' --graph-model --nodal-attention --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0 --no-cuda

DialogueGCN: A Graph Convolutional Neural Network for Emotion Recognition in Conversation. D. Ghosal, N. Majumder, S. Poria, N. Chhaya, & A. Gelbukh. EMNLP-IJCNLP (2019), Hong Kong, China.

# GITHUB ORIGINAL

---
https://github.com/declare-lab/conv-emotion/tree/f34f92eea415465041f7d3f7f8454f88ee9f3acc#dialoguegcn-a-graph-convolutional-neural-network-for-emotion-recognition-in-conversation
